# Figure 1: Host DNA percentage

## A: Human host DNA percentage

Importing libraries

In [2]:
import multiprocessing
import pandas as pd
import subprocess
import pysam
import os
from tqdm import tqdm
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

ModuleNotFoundError: No module named 'seaborn'

Defining functions to parse alignment files

In [2]:
def perChromosome(chr, bam, min_id, min_len):
    resdic = {}
    mapped = 0
    bamfile = pysam.AlignmentFile(bam, "rb")
    reads = bamfile.fetch(chr, multiple_iterators=True)
    for read in reads:
        mismatch = read.get_tag("NM")
        alnLen = read.query_alignment_length
        readLen = read.query_length
        identity = (alnLen - mismatch) / readLen
        if identity >= min_id and alnLen > min_len :
            mapped +=1
    return(mapped)

def getNumberMappedReadsMultiprocess(bam, processes, min_id, min_len):
    try:
        bamfile = pysam.AlignmentFile(bam, "rb")
    except ValueError as e:
        print(e)
    chrs = bamfile.references

    perChromosomePartial = partial(
        perChromosome, bam=bam, min_id = min_id, min_len = min_len)
    p = multiprocessing.Pool(processes)
    result = p.map(perChromosomePartial, chrs)
    p.close()
    p.join()
    mapped = sum(result)
    total = bamfile.mapped + bamfile.unmapped
    return({bam.split("/")[-1].split(".")[0]:[mapped, total]})

def getTotalReads(bam):
    try:
        bamfile = pysam.AlignmentFile(bam, "rb")
    except ValueError as e:
        print(e)
    total = bamfile.mapped + bamfile.unmapped
    return({bam.split("/")[-1].split(".")[0]:total})
        

Setting up the list of bam files

In [ ]:
human_western_bams = 
human_non_western_bams =  